In [1]:
import yahoo_fin.stock_info as si
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from pandas import read_html
#import FunctionFile as FF
from ipynb.fs.full.FunctionFile import RiskVal_func
from ipynb.fs.full.FunctionFile import stock_monte_carlo_func
from ipynb.fs.full.FunctionFile import GrahamInVal_func
from ipynb.fs.full.FunctionFile import WACC_func

nan=np.nan

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

from pandas_datareader import data
from datetime import datetime

In [3]:
stock_list = ['PYPL','AAPL','NVDA','AMD','MSFT']
#stock_list = ['GOOG','AAPL','QCOM','AVGO','PYPL','WIX','OLED','DAL']

#stock_sum = pd.DataFrame(np.array([[nan, nan, nan, nan, nan, nan, nan]]),
stock_sum = pd.DataFrame(np.arange(7*len(stock_list)).reshape((len(stock_list),7)),
                                   columns=['Sticker',
                                            'GrahamInVal($)',
                                            'PrevClose($)',
                                            'Decision', 
                                            'Margin(%)',
                                            'EPS',
                                            'GrowthEst(%)'])

for i in stock_list:
    ami = GrahamInVal_func(i)
    ami1 = pd.DataFrame.from_dict(ami, orient='index')
    ami2 = ami1.transpose()
    #print(ami2)
    stock_sum = stock_sum.append(ami2)

stock_sum.set_index('Sticker', inplace=True)
stock_sum.drop(stock_sum.index[0:len(stock_list)], inplace=True)
stock_sum

,GrahamInVal($),PrevClose($),Decision,Margin(%),EPS,GrowthEst(%)
Sticker,,,,,,
PYPL,126.66,194.57,SELL,53.62,2.18,23.37
AAPL,491.04,462.25,BUY,-5.86,13.19,12.46
NVDA,243.89,490.43,SELL,101.09,5.35,16.83
AMD,42.66,81.66,SELL,91.42,0.51,36.72
MSFT,242.42,210.98,BUY,-12.97,5.76,15


In [4]:
risk_df = RiskVal_func(stock_list,1)

stock_anal1=pd.concat([stock_sum, risk_df], axis=1, sort=False)
stock_anal1

,GrahamInVal($),PrevClose($),Decision,Margin(%),EPS,GrowthEst(%),1yr/95% WC Risk%,1yr/99% WC Risk%
Sticker,,,,,,,,
PYPL,126.66,194.57,SELL,53.62,2.18,23.37,-3.926098,-7.286140
AAPL,491.04,462.25,BUY,-5.86,13.19,12.46,-3.421550,-7.195567
NVDA,243.89,490.43,SELL,101.09,5.35,16.83,-4.275609,-7.764850
AMD,42.66,81.66,SELL,91.42,0.51,36.72,-4.863728,-9.428798
MSFT,242.42,210.98,BUY,-12.97,5.76,15,-3.632843,-6.906204


In [5]:
end = datetime.now()
start = datetime(end.year-1, end.month, end.day)
for x in stock_list:
    globals()[x] = data.DataReader(x,'yahoo',start,end)
        
closing_df = data.DataReader(stock_list,'yahoo',start,end)['Adj Close']
tech_rets = closing_df.pct_change()
rets = tech_rets.dropna()

MC_df = pd.DataFrame(columns = ['start_price','mu','sigma','1yr/99%/q','Var_WC','1Yr/99%/Risk%'], index = stock_list)

days = 365
for i in stock_list:
    MC_df['mu'][i] = rets.mean()[i]
    MC_df['sigma'][i] = rets.std()[i]
    MC_df['start_price'][i] = float(closing_df.tail(1)[i])
        
    runs = 10000
    simulations = np.zeros(runs)

    for run in range(runs):
        simulations[run]=stock_monte_carlo_func(MC_df['start_price'][i],days,MC_df['mu'][i],MC_df['sigma'][i])[days-1]
    
    MC_df['1yr/99%/q'][i] = np.percentile(simulations,1)
    MC_df['Var_WC'][i] = MC_df['1yr/99%/q'][i]-MC_df['start_price'][i]
    MC_df['1Yr/99%/Risk%'][i] = 100*(MC_df['1yr/99%/q'][i]-MC_df['start_price'][i])/MC_df['start_price'][i]

stock_anal2=pd.concat([stock_anal1, MC_df], axis=1, sort=False)
stock_anal2.drop(columns=['mu','sigma','Var_WC'],inplace=True)
stock_anal2

,GrahamInVal($),PrevClose($),Decision,Margin(%),EPS,GrowthEst(%),1yr/95% WC Risk%,1yr/99% WC Risk%,start_price,1yr/99%/q,1Yr/99%/Risk%
Sticker,,,,,,,,,,,
PYPL,126.66,194.57,SELL,53.62,2.18,23.37,-3.926098,-7.286140,192.44,180.41,-6.25105
AAPL,491.04,462.25,BUY,-5.86,13.19,12.46,-3.421550,-7.195567,462.83,438.483,-5.2604
NVDA,243.89,490.43,SELL,101.09,5.35,16.83,-4.275609,-7.764850,485.54,452.188,-6.86904
AMD,42.66,81.66,SELL,91.42,0.51,36.72,-4.863728,-9.428798,81.09,75.0029,-7.50659
MSFT,242.42,210.98,BUY,-12.97,5.76,15,-3.632843,-6.906204,209.7,198.167,-5.49982


In [6]:
WACC_df = pd.DataFrame(columns = ['WACC'], index = stock_list)

for x in stock_list:
    WACC_df['WACC'][x] = WACC_func(x)

stock_anal3=pd.concat([stock_anal2, WACC_df], axis=1, sort=False)
stock_anal3

,GrahamInVal($),PrevClose($),Decision,Margin(%),EPS,GrowthEst(%),1yr/95% WC Risk%,1yr/99% WC Risk%,start_price,1yr/99%/q,1Yr/99%/Risk%,WACC
Sticker,,,,,,,,,,,,
PYPL,126.66,194.57,SELL,53.62,2.18,23.37,-3.926098,-7.286140,192.44,180.41,-6.25105,0.0996676
AAPL,491.04,462.25,BUY,-5.86,13.19,12.46,-3.421550,-7.195567,462.83,438.483,-5.2604,0.0998828
NVDA,243.89,490.43,SELL,101.09,5.35,16.83,-4.275609,-7.764850,485.54,452.188,-6.86904,0.111806
AMD,42.66,81.66,SELL,91.42,0.51,36.72,-4.863728,-9.428798,81.09,75.0029,-7.50659,0.185899
MSFT,242.42,210.98,BUY,-12.97,5.76,15,-3.632843,-6.906204,209.7,198.167,-5.49982,0.0756964
